## Extraktion der Artikel aus gescrapten htmls

In [13]:
import os
import json
from bs4 import BeautifulSoup

data_dir = "../data/"

faz_dir = data_dir + "faz/raw-full-articles/"
zeit_dir = data_dir + "zeit/raw-full-articles/"
sz_dir = data_dir + "sz/raw-full-articles/"

### FAZ
#### Extraktionsfunktion

In [19]:
def extract_full_faz_article(filename):

    with open(faz_dir + filename + ".html", 'r') as file: 
        soup = BeautifulSoup(file, "html.parser")
    # ich habe mich an der html zu folgendem Artikel orientiert: FAS__SD1201503294538183
    text = soup.find("pre",{"class":"text"}).text.strip()
    return text

In [21]:
with open("../data/faz/all-articles.json", "r") as file:
    faz = json.load(file)
    faz = [article for article in faz if ("jemen" in article["title"].lower()) or ("jemen" in article["surtitle"].lower())]

for article in faz:
    id = article["link"].split("uid=")[1]
    text = extract_full_faz_article(id)
    article["id"] = id
    article["text"]= text



In [23]:
with open ("../data/faz/all-relevant-fulltext.json", "w") as file:
    json.dump(faz, file, indent = 2)

### ZEIT
#### Extraktionsfunktion

In [24]:
def extract_full_zeit_article(filename):

    with open(zeit_dir + filename + ".html", 'r') as file: 
        soup = BeautifulSoup(file, "html.parser")
    # ich habe mich an der html zu folgendem Artikel orientiert: view-source:https://www.wiso-net.de//document/ZEIO__5DCBDAE4A902F6F8EE277E8578808207/hitlist/475?all=
    text = soup.find("pre",{"class":"text"}).text.strip()
    # Sieht so aus: <pre class="boldMedium">
    abstract = soup.find("pre",{"class":"boldMedium"}).text.strip()
    try:
        #<pre class="italic">Theo Sommer</pre>
        author = soup.find("pre",{"class":"italic"}).text.strip()
    except:
        author =""

    return text, abstract, author


In [26]:
with open("../data/zeit/all-articles.json", "r") as file:
    zeit = json.load(file)
    zeit = [article for article in zeit if ("jemen" in article["title"].lower()) or ("jemen" in article["surtitle"].lower())]

for article in zeit:
    id = article["link"].split("document/")[1].split("/")[0]
    text, abstract, author = extract_full_zeit_article(id)
    # id wird als value mit dem key id in dictionary gespeichert
    article["id"] = id
    article["text"]= text
    article["abstract"]=abstract
    article["author"]=author

Ausführen der Extraktion und Abspeichern

In [ ]:
files = os.listdir(zeit_dir)
zeit_full_articles = []

for file in files:
    zeit_full_articles = zeit_full_articles + extract_full_zeit_article(file)

with open(data_dir + "zeit/full-articles.json", "w") as file:
    json.dump(zeit_full_articles, file, indent = 2)

### SZ
#### Extraktionsfunktion

In [ ]:
def extract_full_sz_articles(filename):

    with open(sz_dir + filename, 'r') as file: 
        soup = BeautifulSoup(file, "html.parser")

#  div[class*]... war das Trennzechen, bei dem ein neuer Artikel begann, brauchen wir das hier?
    full_articles_html = soup.select("div[class*='article-row clearfix']") 
    full_articles_list = []

# ich habe mich an der html zu folgendem Artikel orientiert: https://archiv.szarchiv.de/Portal/restricted/Start.act?articleId=A70260928
    for article in full_articles_html:
        # Sieht so aus: <title>SZ LibraryNet - Volltext Bis zum letzten   Reiskorn (A70260928)</title>
        title =  article.find_all("title").text.split("Volltext ")[1].split(" (")[0].strip
        # Es gibt auch die Option, hier einen Perma-Link einzutragen. Sieht so aus: <pre class="text permaLink"><strong>Dauerhafte Adresse des Dokuments: </strong><a href="https://www.wiso-net.de/document/ZEIO__932344341ad0602bd3d227a8d80550820d8209ba">https://www.wiso-net.de/document/ZEIO__932344341ad0602bd3d227a8d80550820d8209ba</a></pre>
        link = "https://archiv.szarchiv.de/Portal/restricted/Start.act?articleId="  + str(filename)
        # Sieht so aus: 	
			#<div class="fullTextLeftTop">
					#<span>18.03.2017</span>
        date = article.find("div",{"class":"fullTextLeftTop"}).text.strip
        # Sieht so aus: <SZ.UT><P><FONT FACE="Arial, Helvetica">Jemens Kriegsparteien nutzen den Mangel als Waffe. <BR />Dringend notwendige Hilfe wird gnadenlos blockiert</FONT></P></SZ.UT>
        abstract = ?
        # Sieht super strange aus: <P><FONT FACE="Arial, Helvetica"><B>Washington &ndash;</B> F&uuml;r US-Pr&auml;sident Donald Trump geht es um die nationale Sicherheit und um Terrorbek&auml;mpfung. Mit seinem tempor&auml;ren Einreisestopp will er verhindern, dass Menschen aus L&auml;ndern mit &bdquo;terroristischem Hintergrund&ldquo;, in denen die &bdquo;Sicherheits&uuml;berpr&uuml;fung&ldquo; nicht gew&auml;hrleistet sei, in die USA reisen. F&uuml;r Richter Derrick Watson vom Bundesgericht in Hawaii hingegen, der Trumps Einreisestopp am Mittwochabend vorl&auml;ufig gestoppt hat, geht es um die Verfassung der Vereinigten Staaten. Der Einreisestopp, so Watson, diskriminiere eine ganze Religion und sei deshalb verfassungswidrig. </FONT></P>
            #<P><FONT FACE="Arial, Helvetica">Es ist das zweite Mal,
        text = article.find("pre",{"class":"text"}).text.strip()
        # Steht manchmal oben: <P ALIGN="CENTER"><FONT FACE="Arial, Helvetica"><AUTHOR><B>VON <AUTHORNAME>PAUL-ANTON KR&Uuml;GER</AUTHORNAME></B></AUTHOR></FONT></P>
        # Und manchmal unten als Kürzel: <P><FONT FACE="sans-serif" SIZE="-1"><AUTHOR>SZ.de/rtr</AUTHOR></FONT></P>
        # Und manchmal unten rechts: <P ALIGN="RIGHT"><FONT FACE="Arial, Helvetica"><AUTHOR><B><AUTHORNAME>SACHA BATTHYANY</AUTHORNAME> </B></AUTHOR></FONT></P>
        # Und manchmal am Anfang in kursiv: <author>Analyse von <authorname>Kathleen Hildebrand</authorname></author>
        author = article.find("author").text.split("de/")[1].strip
        # wenn es einen gibt: <SZ.DT><P><FONT FACE="sans-serif"><B>Migration</B></FONT></P></SZ.DT> 
        surtitle = article.find_all("pre",{"class":"text"}).text


        full_articles_list.append({"title":title, "link":link, "date":date, "abstract":abstract, "author":author, "surtitle":surtitle, "source":"ZEIT", "text": text})
    return full_articles_list

Ausführen der Extraktion und Abspeichern

In [ ]:
files = os.listdir(sz_dir)
sz_full_articles = []

for file in files:
    sz_full_articles = sz_full_articles + extract_full_sz_articles(file)

with open(data_dir + "sz/full-articles.json", "w") as file:
    json.dump(sz_full_articles, file, indent = 2)